# Dino Game using Google Mediapipe for hand-tracking

    Saad Bazaz  - i180621
    Abraar Raza - i1    

To act as a control, so we can demonstrate the effectiveness of our own hand-tracking algorithm.

## Installations

In [1]:
# !pip install mediapipe

In [2]:
# !pip install opencv-python

## Imports

In [3]:
# For hand-tracking
import mediapipe as mp
import cv2
import numpy as np

In [4]:
# For Game Emulation
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import time

## Initialize

### Hand-Tracking Algorithm

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

### Game Environment

In [6]:
## Initialize Selenium Browser

# https://sites.google.com/a/chromium.org/chromedriver/home
# Download ChromeDriver and extract. Then enter the fullpath here.
PATH_TO_CHROME = r"/Users/saadbazaz/Documents/FAST Studies/Digital Image Processing Lab/Project/Dino-Game-Hand-Gestures/t-rex-runner-gh-pages/chromedriver"

# The link to your Dino Game's index.html
DINO_GAME_LINK = r"/Users/saadbazaz/Documents/FAST Studies/Digital Image Processing Lab/Project/Dino-Game-Hand-Gestures/t-rex-runner-gh-pages/index.html"


browser = webdriver.Chrome(PATH_TO_CHROME)
res = browser.get('file://' + DINO_GAME_LINK)

In [7]:
## Initialize Selenium Action Chains

actions = ActionChains(browser)

In [8]:
## Test Actions


# actions.key_down(Keys.DOWN)
# actions.perform()

# time.sleep(1)



# actions.key_up(Keys.DOWN)
# actions.perform()

# time.sleep(1)

# actions.send_keys(Keys.SPACE)
# actions.perform()

# Run Game

In [9]:
# define a video capture object
vid = cv2.VideoCapture(0)

upper_thresh = 0.3
lower_thresh = 0.7

# 0 for Keep Running, 1 is for Jump, 2 is for Duck

prev_pos = 0


ret, frame = vid.read()

scale_percent = 25# percent of original size
width = int(frame.shape[1] * scale_percent / 100)
height = int(frame.shape[0] * scale_percent / 100)
dim = (width, height)


with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    while vid.isOpened():

        # Capture the video frame
        # by frame
        ret, frame = vid.read()
        

        frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        
        results = holistic.process(image)
        
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)        
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        

        if results.right_hand_landmarks is not None:
            pos_y = results.right_hand_landmarks.landmark[10].y

            ## Draw threshold lines
            y_pix = int(np.multiply(lower_thresh, image.shape[0]))
            cv2.line(image, (0, y_pix), (image.shape[1],  y_pix), (0,255,0), thickness=2)
            
            
            y_pix = int(np.multiply(upper_thresh, image.shape[0]))
            cv2.line(image, (0, y_pix), (image.shape[1],  y_pix), (255,0,0), thickness=2)
    

            if pos_y < upper_thresh:
                # print ("Jump!")
                               
            
                if prev_pos == 1:
                    pass  
                elif prev_pos == 2:
                    actions = ActionChains(browser)                    
                    actions.key_up(Keys.DOWN)
                    actions.send_keys(Keys.SPACE)
                    actions.perform() 
                else:
                    actions = ActionChains(browser)
                    actions.send_keys(Keys.SPACE)
                    actions.perform()                 
            
                prev_pos = 1
    
            elif pos_y > lower_thresh:
                # print ("Duck.")

                if prev_pos == 1:
                    actions = ActionChains(browser)
                    actions.key_down(Keys.DOWN)
                    actions.perform()     
                elif prev_pos == 2:
                    pass
                else:
                    actions = ActionChains(browser)
                    actions.key_down(Keys.DOWN)
                    actions.perform()                     

                prev_pos = 2


            else:
                # print ("Keep running.")

                if prev_pos == 2:
                    actions = ActionChains(browser)                    
                    actions.key_up(Keys.DOWN)
                    actions.perform() 

                prev_pos = 0            

        # Display the resulting frame
        cv2.imshow('frame', image)

        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()
for i in range (1,5):
    cv2.waitKey(1)